# Prepare emissions files

Here we take the ssp245 scenario as a baseline, and modify the emissions based on each policy option. We care about temperature deltas in each case. Therefore underlying SSP choice will have some effect due to temperature impacts on carbon cycle and atmsopheric chemistry feedbacks, but not likely to be very large.

In [ ]:
import copy

from climateforcing.utils import mkdir_p
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

## Create baseline scenarios

SSP2-4.5, plus modifications thereof

In [ ]:
emissions_out = {}
scenarios = ["ssp245", "baseline", "hfc-32", "propane", "propane-as-slcf"]

In [ ]:
ssp_df = pd.read_csv('../data_input_large/rcmip-emissions-annual-means-v5-1-0.csv')

In [ ]:
ssp_df.loc[
    (ssp_df['Model']=='MESSAGE-GLOBIOM')&
    (ssp_df['Region']=='World')&
    (ssp_df['Scenario']=='ssp245')&
    (ssp_df['Variable'].str.endswith('|HFC125')),'2015':'2110':5
]

In [ ]:
ssp_df.loc[
    (ssp_df['Model']=='MESSAGE-GLOBIOM')&
    (ssp_df['Region']=='World')&
    (ssp_df['Scenario']=='ssp245')&
    (ssp_df['Variable'].str.endswith('|HCFC22')),'2015':'2110':5
]

In [ ]:
ssp_df.loc[
    (ssp_df['Model']=='MESSAGE-GLOBIOM')&
    (ssp_df['Region']=='World')&
    (ssp_df['Scenario']=='ssp245')&
    (ssp_df['Variable'].str.endswith('|HFC32')),'2015':'2110':5
]

In [ ]:
ssp_df.loc[
    (ssp_df['Model']=='MESSAGE-GLOBIOM')&
    (ssp_df['Region']=='World')&
    (ssp_df['Scenario']=='ssp245')&
    (ssp_df['Variable'].str.endswith('|VOC')),'2015':'2110':5
]

In [ ]:
years = np.arange(1750, 2111)

startyear = 1750
first_scenyear = 2015
last_scenyear = 2110
first_row = int(first_scenyear-startyear)
last_row = int(last_scenyear-startyear)

species = [  # in fair 1.6, order is important
    '|CO2|MAGICC Fossil and Industrial',
    '|CO2|MAGICC AFOLU',
    '|CH4',
    '|N2O',
    '|Sulfur',
    '|CO',
    '|VOC', # 7
    '|NOx',
    '|BC',
    '|OC',
    '|NH3',
    '|CF4',
    '|C2F6',
    '|C6F14',
    '|HFC23',
    '|HFC32',  # 16
    '|HFC4310mee',
    '|HFC125', # 18
    '|HFC134a',
    '|HFC143a',
    '|HFC227ea',
    '|HFC245fa',
    '|SF6',
    '|CFC11',
    '|CFC12',
    '|CFC113',
    '|CFC114',
    '|CFC115',
    '|CCl4',
    '|CH3CCl3',
    '|HCFC22',  # 31
    '|HCFC141b',
    '|HCFC142b',
    '|Halon1211',
    '|Halon1202',
    '|Halon1301',
    '|Halon2402',
    '|CH3Br',
    '|CH3Cl',
]

unit_convert = np.ones(40)
unit_convert[1] = 12/44/1000
unit_convert[2] = 12/44/1000
unit_convert[4] = 28/44/1000
unit_convert[5] = 32/64
unit_convert[8] = 14/46

for scenario in scenarios:
    emissions_out[scenario] = np.ones((361, 40)) * np.nan
    emissions_out[scenario][:,0] = years

    years_future = [2015] + list(range(2020,2111,10))
    for i, specie in enumerate(species):
        emissions_out[scenario][:first_row,i+1] = ssp_df.loc[
            (ssp_df['Model']=='MESSAGE-GLOBIOM')&
            (ssp_df['Region']=='World')&
            (ssp_df['Scenario']=='ssp245')&
            (ssp_df['Variable'].str.endswith(specie)),str(startyear):'2014']*unit_convert[i+1]
        f = interp1d(years_future, ssp_df.loc[
            (ssp_df['Model']=='MESSAGE-GLOBIOM')&
            (ssp_df['Region']=='World')&
            (ssp_df['Scenario']=='ssp245')&
            (ssp_df['Variable'].str.endswith(specie)),'2015':'2110'].dropna(axis=1))
        emissions_out[scenario][first_row:(last_row+1), i+1] = f(
            np.arange(first_scenyear, last_scenyear+1)
        )*unit_convert[i+1]

## Modify the emissions of the HFC modification scenarios

In [ ]:
df_ac = pd.read_excel('../data_input/Split-AC_sector_emissions.xlsx')

In [ ]:
year = np.array(df_ac.iloc[4, 3:].values, dtype=float)

In [ ]:
year

In [ ]:
df_ac.iloc[5:9, ]

In [ ]:
hcfc22 = np.array(df_ac.iloc[5, 3:].values, dtype=float)
hfc125 = np.array(df_ac.iloc[6, 3:].values, dtype=float)
hfc32 = np.array(df_ac.iloc[7, 3:].values, dtype=float)
c3h8 = np.array(df_ac.iloc[8, 3:].values, dtype=float)

In [ ]:
f = interp1d(year, hcfc22)
emissions_out['baseline'][267:351, 31] = f(np.arange(2017, 2101))
f = interp1d(year, hfc125)
emissions_out['baseline'][267:351, 18] = f(np.arange(2017, 2101))
f = interp1d(year, hfc32)
emissions_out['baseline'][267:351, 16] = f(np.arange(2017, 2101))
#f = interp1d(year, c3h8)
#emissions_out['baseline'][267:351] = f(np.arange(2017, 2101))

In [ ]:
df_ac.iloc[15:19, ]

In [ ]:
hcfc22 = np.array(df_ac.iloc[15, 3:].values, dtype=float)
hfc125 = np.array(df_ac.iloc[16, 3:].values, dtype=float)
hfc32 = np.array(df_ac.iloc[17, 3:].values, dtype=float)
c3h8 = np.array(df_ac.iloc[18, 3:].values, dtype=float)

In [ ]:
f = interp1d(year, hcfc22)
emissions_out['hfc-32'][267:351, 31] = f(np.arange(2017, 2101))
f = interp1d(year, hfc125)
emissions_out['hfc-32'][267:351, 18] = f(np.arange(2017, 2101))
f = interp1d(year, hfc32)
emissions_out['hfc-32'][267:351, 16] = f(np.arange(2017, 2101))
#f = interp1d(year, c3h8)
#emissions_out['baseline'][267:351] = f(np.arange(2017, 2101))

In [ ]:
df_ac.iloc[25:29, ]

In [ ]:
hcfc22 = np.array(df_ac.iloc[25, 3:].values, dtype=float)
hfc125 = np.array(df_ac.iloc[26, 3:].values, dtype=float)
hfc32 = np.array(df_ac.iloc[27, 3:].values, dtype=float)
c3h8 = np.array(df_ac.iloc[28, 3:].values, dtype=float)

In [ ]:
f = interp1d(year, hcfc22)
emissions_out['propane'][267:351, 31] = f(np.arange(2017, 2101))
emissions_out['propane-as-slcf'][267:351, 31] = f(np.arange(2017, 2101))
f = interp1d(year, hfc125)
emissions_out['propane'][267:351, 18] = f(np.arange(2017, 2101))
emissions_out['propane-as-slcf'][267:351, 18] = f(np.arange(2017, 2101))
f = interp1d(year, hfc32)
emissions_out['propane'][267:351, 16] = f(np.arange(2017, 2101))
emissions_out['propane-as-slcf'][267:351, 16] = f(np.arange(2017, 2101))

f = interp1d(year, c3h8 / 1000)  # original dataset is kt/yr, fair input is Mt/yr
# from correspondence with Steve Smith, all VOC emissions are aggregated and there does not seem to be any
# weighting for chemical effect, radiative efficiency, etc. so we can just add C3H8 emissions to total VOCs
# in the counterfactual
emissions_out['propane-as-slcf'][267:351, 7] = f(np.arange(2017, 2101)) + emissions_out['ssp245'][267:351, 7]

In [ ]:
pl.plot(np.arange(2010, 2101), emissions_out['ssp245'][260:351, 31])
pl.plot(np.arange(2010, 2101), emissions_out['baseline'][260:351, 31])
pl.plot(np.arange(2010, 2101), emissions_out['hfc-32'][260:351, 31])
pl.plot(np.arange(2010, 2101), emissions_out['propane'][260:351, 31])
pl.plot(np.arange(2010, 2101), emissions_out['propane-as-slcf'][260:351, 31])
pl.title('HCFC22 emissions')

In [ ]:
pl.plot(np.arange(2010, 2101), emissions_out['ssp245'][260:351, 18])
pl.plot(np.arange(2010, 2101), emissions_out['baseline'][260:351, 18])
pl.plot(np.arange(2010, 2101), emissions_out['hfc-32'][260:351, 18])
pl.plot(np.arange(2010, 2101), emissions_out['propane'][260:351, 18])
pl.plot(np.arange(2010, 2101), emissions_out['propane-as-slcf'][260:351, 18])
pl.title('HFC125 emissions')

In [ ]:
pl.plot(np.arange(2010, 2101), emissions_out['ssp245'][260:351, 16])
pl.plot(np.arange(2010, 2101), emissions_out['baseline'][260:351, 16])
pl.plot(np.arange(2010, 2101), emissions_out['hfc-32'][260:351, 16])
pl.plot(np.arange(2010, 2101), emissions_out['propane'][260:351, 16])
pl.plot(np.arange(2010, 2101), emissions_out['propane-as-slcf'][260:351, 16])
pl.title('HFC32 emissions')

In [ ]:
pl.plot(np.arange(2010, 2101), emissions_out['ssp245'][260:351, 7])
pl.plot(np.arange(2010, 2101), emissions_out['baseline'][260:351, 7])
pl.plot(np.arange(2010, 2101), emissions_out['hfc-32'][260:351, 7])
pl.plot(np.arange(2010, 2101), emissions_out['propane'][260:351, 7])
pl.plot(np.arange(2010, 2101), emissions_out['propane-as-slcf'][260:351, 7])
pl.title('VOC emissions')

In [ ]:
mkdir_p('../data_output/fair_emissions_files/')
for scenario in scenarios:
    np.savetxt('../data_output/fair_emissions_files/{}.csv'.format(scenario), emissions_out[scenario], delimiter=',')